In [ ]:
import pandas as pd
import numpy as np
import os
import re

# ==============================================================================
# HÀM CHUẨN HÓA VÀ HÀM TRỢ GIÚP
# ==============================================================================

def normalize_text(text):
    """
    Chuẩn hóa văn bản bằng cách loại bỏ nội dung bên trong cặp dấu ngoặc đơn,
    đảm bảo chỉ có một khoảng trắng giữa các từ và chuyển thành chữ thường.
    """
    if pd.isna(text):
        return ""
    cleaned_text = re.sub(r'\s*\(.*?\)\s*', ' ', str(text))
    final_text = re.sub(r'\s+', ' ', cleaned_text).strip().lower()
    return final_text

def add_normalized_keys(df: pd.DataFrame) -> pd.DataFrame:
    """
    Thêm các cột khóa đã được chuẩn hóa ('key_title', 'key_acronym') vào DataFrame.
    """
    df_copy = df.copy()
    if 'title' in df_copy.columns:
        df_copy['key_title'] = df_copy['title'].apply(normalize_text)
    else:
        print("Cảnh báo: DataFrame không có cột 'title'. Không thể tạo key_title.")
        df_copy['key_title'] = ''

    if 'acronym' in df_copy.columns:
        df_copy['key_acronym'] = df_copy['acronym'].apply(normalize_text)
    else:
        df_copy['key_acronym'] = ''
        
    return df_copy

def load_non_link_acronyms(filepath: str) -> set:
    """
    Đọc file TXT chứa các acronym của conference non-link.
    """
    non_link_acronyms = set()
    if not os.path.exists(filepath):
        print(f"Cảnh báo: File non-link acronym không tồn tại tại '{filepath}'. Bỏ qua bước lọc này.")
        return non_link_acronyms
    try:
        with open(filepath, 'r', encoding='utf-8') as f:
            for line in f:
                acronym = line.strip()
                if acronym:
                    non_link_acronyms.add(normalize_text(acronym))
        print(f"Đã tải {len(non_link_acronyms)} acronym non-link từ '{filepath}'.")
    except Exception as e:
        print(f"Lỗi khi đọc file non-link acronym '{filepath}': {e}")
    return non_link_acronyms

# ==============================================================================
# HÀM TÍNH TỔNG SỐ CONFERENCE DUY NHẤT
# ==============================================================================

def calculate_and_print_total_unique_count(all_files: list):
    """
    Đọc tất cả các file, tính toán và in ra tổng số conference duy nhất
    dựa trên title và acronym đã được chuẩn hóa.
    """
    total_unique_conferences = set()
    print("\n" + "="*80)
    print(" BƯỚC 0: TÍNH TOÁN TỔNG SỐ CONFERENCE DUY NHẤT TRÊN TẤT CẢ CÁC FILE")
    print("="*80)

    for file_path in all_files:
        try:
            df = pd.read_csv(file_path, encoding='utf-8-sig', na_values=['', ' '], dtype=str, low_memory=False)
            df_normalized = add_normalized_keys(df)

            if 'key_title' not in df_normalized.columns or 'key_acronym' not in df_normalized.columns:
                print(f"Cảnh báo: Bỏ qua file {os.path.basename(file_path)} vì không thể tạo cột khóa.")
                continue
            
            unique_keys_in_file = set(zip(df_normalized['key_title'], df_normalized['key_acronym']))
            total_unique_conferences.update(unique_keys_in_file)
            print(f"Đã xử lý file: {os.path.basename(file_path)}")

        except Exception as e:
            print(f"Lỗi đọc file {os.path.basename(file_path)} trong quá trình đếm, sẽ bỏ qua file này: {e}")

    print("\n" + "-"*40)
    print(f">>> TỔNG SỐ CONFERENCE DUY NHẤT (dựa trên title và acronym đã chuẩn hóa) trong tất cả các file là: {len(total_unique_conferences)}")
    print("-"*40)

# ==============================================================================
# HÀM XỬ LÝ CHÍNH (CHỈ DỰA VÀO CỘT 'Note')
# ==============================================================================

def process_files_by_note(
    all_input_files: list,
    aggregated_correct_path: str,
    aggregated_recrawl_path: str,
    non_link_acronyms_file: str = None
):
    """
    Hàm xử lý chính, chỉ dựa vào cột 'Note' để phân loại đúng/sai.
    """
    print("\n" + "="*80)
    print(" BẮT ĐẦU KỊCH BẢN: PHÂN LOẠI CHỈ DỰA VÀO CỘT 'Note'")
    print("="*80 + "\n")

    all_dfs = []

    # --- BƯỚC 1: ĐỌC VÀ GỘP TẤT CẢ CÁC FILE ---
    print("--- BƯỚC 1: Đọc và gộp tất cả các file đầu vào ---")
    for file_path in all_input_files:
        try:
            df = pd.read_csv(file_path, encoding='utf-8-sig', na_values=['', ' '], dtype=str, low_memory=False)
            df = add_normalized_keys(df)
            all_dfs.append(df)
            print(f"Đã đọc thành công file: {os.path.basename(file_path)}")
        except Exception as e:
            print(f"Lỗi đọc file {file_path}: {e}")

    if not all_dfs:
        print("Không có dữ liệu để xử lý. Dừng chương trình.")
        return

    master_df = pd.concat(all_dfs, ignore_index=True)
    print(f"\nĐã gộp thành công. Tổng số dòng ban đầu: {len(master_df)}")

    # --- BƯỚC 2: PHÂN LOẠI DỰA TRÊN CỘT 'Note' ---
    print("\n--- BƯỚC 2: Phân loại dữ liệu dựa trên sự tồn tại của nội dung trong cột 'Note' ---")
    
    # Mặc định tất cả là đúng, sau đó xác định các dòng cần recrawl
    if 'Note' in master_df.columns:
        # Điều kiện để một dòng cần recrawl là cột 'Note' không rỗng (notna) và không phải là chuỗi trống
        is_recrawl = master_df['Note'].notna() & (master_df['Note'].astype(str).str.strip() != '')
    else:
        # Nếu không có cột 'Note', không có gì để đưa vào recrawl
        is_recrawl = pd.Series([False] * len(master_df), index=master_df.index)
        print("Cảnh báo: Không tìm thấy cột 'Note' trong dữ liệu tổng hợp.")

    recrawl_df = master_df[is_recrawl].copy()
    correct_df = master_df[~is_recrawl].copy()

    print(f"Số dòng được xác định là 'Đúng' (Note trống): {len(correct_df)}")
    print(f"Số dòng được xác định là 'Cần Recrawl' (Note có nội dung): {len(recrawl_df)}")

    # --- BƯỚC 3: LỌC BỎ CÁC ACRONYM NON-LINK KHỎI DANH SÁCH RECRAWL ---
    print("\n--- BƯỚC 3: Lọc bỏ các acronym non-link khỏi danh sách recrawl ---")
    if non_link_acronyms_file:
        non_link_acronyms = load_non_link_acronyms(non_link_acronyms_file)
        if non_link_acronyms and not recrawl_df.empty:
            initial_recrawl_count = len(recrawl_df)
            recrawl_df = recrawl_df[
                ~recrawl_df['key_acronym'].isin(non_link_acronyms)
            ].copy()
            print(f"Đã loại bỏ {initial_recrawl_count - len(recrawl_df)} dòng có acronym non-link khỏi danh sách recrawl.")

    # --- BƯỚC 4: LƯU FILE CUỐI CÙNG (LOẠI BỎ TRÙNG LẶP) ---
    print("\n--- BƯỚC 4: Hoàn tất và lưu file tổng hợp cuối cùng ---")
    subset_keys = ['key_title', 'key_acronym']
    key_cols_to_drop = ['key_title', 'key_acronym']

    if not correct_df.empty:
        # Giữ lại bản ghi đầu tiên cho mỗi conference được coi là đúng
        final_correct_df = correct_df.drop_duplicates(subset=subset_keys, keep='first').drop(columns=key_cols_to_drop, errors='ignore')
        final_correct_df.to_csv(aggregated_correct_path, index=False, encoding='utf-8-sig')
        print(f"Đã lưu {len(final_correct_df)} dòng ĐÚNG DUY NHẤT vào '{aggregated_correct_path}'")
    
    if not recrawl_df.empty:
        # Giữ lại bản ghi đầu tiên cho mỗi conference cần recrawl
        final_recrawl_df = recrawl_df.drop_duplicates(subset=subset_keys, keep='first').drop(columns=key_cols_to_drop, errors='ignore')
        final_recrawl_df.to_csv(aggregated_recrawl_path, index=False, encoding='utf-8-sig')
        print(f"Đã lưu {len(final_recrawl_df)} dòng CẦN RECRAWL DUY NHẤT vào '{aggregated_recrawl_path}'")
    else:
        print(f"Không có dòng nào cần recrawl sau khi lọc. File '{aggregated_recrawl_path}' sẽ không được tạo hoặc sẽ trống.")


# ==============================================================================
# KHU VỰC THỰC THI
# ==============================================================================
if __name__ == "__main__":
    # --- CẤU HÌNH ---
    input_csv_files = [
        './src/conference/evaluate/batch2.csv',
        './src/conference/evaluate/batch3.csv',
        './src/conference/evaluate/batch8.csv',
        './src/conference/evaluate/batch12.csv',
        './src/conference/evaluate/batch13.csv',
        './src/conference/evaluate/batch16.csv',
        './src/conference/evaluate/batch19.csv',
        './src/conference/evaluate/evaluate_recrawl_batch_2_3_lan_1.csv',
        './src/conference/evaluate/evaluate_recrawl_batch_8_lan_1.csv',
        './src/conference/evaluate/evaluate_recrawl_batch_12_lan_1.csv',
        './src/conference/evaluate/evaluate_recrawl_batch_13_lan_1.csv',
        './src/conference/evaluate/evaluate_recrawl_batch_16_lan_1.csv',
        './src/conference/evaluate/evaluate_recrawl_batch_19_lan_1.csv',
        './src/conference/evaluate/evaluate_recrawl_all_tung_1_50.csv',
        './src/conference/evaluate/evaluate_recrawl_all_tung_51_100.csv',
        './src/conference/evaluate/evaluate_recrawl_all_tung_101_150.csv',
        './src/conference/evaluate/evaluate_recrawl_all_tung_151_159.csv',
        './src/conference/evaluate/evaluate_recrawl_all_tung_1_50_lan_2.csv',
        './src/conference/evaluate/evaluate_recrawl_all_tung_51_100_lan_2.csv',
        './src/conference/evaluate/evaluate_recrawl_all_tung_101_139_lan_2.csv',
        './src/conference/evaluate/evaluate_recrawl_all_tung_lan_3.csv',
        './src/conference/evaluate/evaluate_recrawl_all_tung_1_50_lan_4.csv',
        './src/conference/evaluate/evaluate_crawl_not_crawl_1_50.csv',
        './src/conference/evaluate/evaluate_crawl_not_crawl_51_end.csv',
    
    ]
    recrawled_results_files = [
        './src/conference/evaluate/evaluate_recrawl_batch_2_3_lan_1.csv',
        './src/conference/evaluate/evaluate_recrawl_batch_8_lan_1.csv',
        './src/conference/evaluate/evaluate_recrawl_batch_12_lan_1.csv',
        './src/conference/evaluate/evaluate_recrawl_batch_13_lan_1.csv',
        './src/conference/evaluate/evaluate_recrawl_batch_16_lan_1.csv',
        './src/conference/evaluate/evaluate_recrawl_batch_19_lan_1.csv',
        './src/conference/evaluate/evaluate_recrawl_all_tung_1_50.csv',
        './src/conference/evaluate/evaluate_recrawl_all_tung_51_100.csv',
        './src/conference/evaluate/evaluate_recrawl_all_tung_101_150.csv',
        './src/conference/evaluate/evaluate_recrawl_all_tung_151_159.csv',
        './src/conference/evaluate/evaluate_recrawl_all_tung_1_50_lan_2.csv',
        './src/conference/evaluate/evaluate_recrawl_all_tung_51_100_lan_2.csv',
        './src/conference/evaluate/evaluate_recrawl_all_tung_101_139_lan_2.csv',
        './src/conference/evaluate/evaluate_recrawl_all_tung_lan_3.csv',
        './src/conference/evaluate/evaluate_recrawl_all_tung_1_50_lan_4.csv',
        './src/conference/evaluate/evaluate_crawl_not_crawl_1_50.csv',
        './src/conference/evaluate/evaluate_crawl_not_crawl_51_end.csv',
    ]
    
    # Gộp hai danh sách file lại thành một để xử lý
    all_files_to_process = recrawled_results_files + input_csv_files

    aggregated_correct_file = './src/conference/evaluate/ALL_BATCHES_correct_by_note.csv'
    aggregated_recrawl_file = './src/conference/evaluate/ALL_BATCHES_recrawl_by_note.csv'
    
    non_link_acronyms_txt_file = './non_link.txt'

    # --- BƯỚC 0: TÍNH TOÁN VÀ IN RA TỔNG SỐ CONFERENCE DUY NHẤT ---
    calculate_and_print_total_unique_count(all_files_to_process)

    # --- Chạy kịch bản xử lý chính ---
    process_files_by_note(
        all_input_files=all_files_to_process,
        aggregated_correct_path=aggregated_correct_file,
        aggregated_recrawl_path=aggregated_recrawl_file,
        non_link_acronyms_file=non_link_acronyms_txt_file
    )

In [2]:
import pandas as pd
import os
import re

# ==============================================================================
# HÀM CHUẨN HÓA VÀ HÀM TRỢ GIÚP (Giữ nguyên)
# ==============================================================================

def normalize_text(text):
    if pd.isna(text):
        return ""
    cleaned_text = re.sub(r'\s*\(.*?\)\s*', ' ', str(text))
    final_text = re.sub(r'\s+', ' ', cleaned_text).strip().lower()
    return final_text

def add_normalized_keys(df: pd.DataFrame) -> pd.DataFrame:
    df_copy = df.copy()
    df_copy['key_title'] = df_copy['title'].apply(normalize_text) if 'title' in df_copy.columns else ''
    df_copy['key_acronym'] = df_copy['acronym'].apply(normalize_text) if 'acronym' in df_copy.columns else ''
    return df_copy

# ==============================================================================
# HÀM ĐẾM SỐ LƯỢNG DUY NHẤT (Giữ nguyên)
# ==============================================================================

def calculate_and_print_total_unique_count(all_files: list):
    total_unique_conferences = set()
    print("\n" + "="*80)
    print(" BƯỚC 0: TÍNH TOÁN TỔNG SỐ CONFERENCE DUY NHẤT TRÊN TẤT CẢ CÁC FILE")
    print("="*80)
    for file_path in all_files:
        try:
            df = pd.read_csv(file_path, encoding='utf-8-sig', na_values=['', ' '], dtype=str, low_memory=False)
            df_normalized = add_normalized_keys(df)
            if not df_normalized.empty:
                unique_keys_in_file = set(zip(df_normalized['key_title'], df_normalized['key_acronym']))
                total_unique_conferences.update(unique_keys_in_file)
                print(f"Đã xử lý file để đếm: {os.path.basename(file_path)}")
        except Exception as e:
            print(f"Lỗi đọc file {os.path.basename(file_path)} trong quá trình đếm: {e}")
    print("\n" + "-"*40)
    print(f">>> TỔNG SỐ CONFERENCE DUY NHẤT (dựa trên title và acronym đã chuẩn hóa) là: {len(total_unique_conferences)}")
    print("-"*40)

# ==============================================================================
# HÀM XỬ LÝ CHÍNH THEO LOGIC MỚI
# ==============================================================================
def process_by_any_correct_version(
    input_files: list,
    aggregated_correct_path: str,
    aggregated_recrawl_path: str,
    recrawled_files: list = None
):
    """
    Phân loại conference: nếu có bất kỳ phiên bản nào không có 'note',
    nó sẽ được coi là 'correct' và không nằm trong danh sách 'recrawl'.
    Luôn ưu tiên lấy phiên bản mới nhất.
    """
    print("\n" + "="*80)
    print(" BẮT ĐẦU KỊCH BẢN: ƯU TIÊN BẤT KỲ PHIÊN BẢN ĐÚNG NÀO")
    print("="*80 + "\n")

    # --- BƯỚC 1: ĐỌC VÀ GỘP TẤT CẢ CÁC FILE (ƯU TIÊN FILE ĐÃ RECRAWL) ---
    print("--- BƯỚC 1: Đọc và gộp tất cả các file ---")
    all_dfs = []
    all_files_in_order = (recrawled_files or []) + (input_files or [])
    
    for file_path in all_files_in_order:
        try:
            df = pd.read_csv(file_path, encoding='utf-8-sig', na_values=[''], dtype=str).fillna('')
            df = add_normalized_keys(df)
            df = df[df['key_title'] != ''].copy()
            if not df.empty:
                all_dfs.append(df)
        except Exception as e:
            print(f"Lỗi xử lý file {file_path}: {e}")
            continue
            
    if not all_dfs:
        print("Không có dữ liệu để xử lý. Dừng chương trình.")
        return

    master_df = pd.concat(all_dfs, ignore_index=True)
    print(f"Đã gộp thành công. Tổng số dòng để xử lý: {len(master_df)}")

    # --- BƯỚC 2: XÁC ĐỊNH DANH SÁCH CÁC CONFERENCE "ĐÚNG" ---
    print("\n--- BƯỚC 2: Xác định danh sách các conference có ít nhất một phiên bản đúng ---")
    key_cols = ['key_title', 'key_acronym']
    
    # Điều kiện một dòng được coi là "đúng"
    is_correct_condition = ~master_df['note'].notna() | (master_df['note'].astype(str).str.strip() == '')
    
    # Lấy tất cả các dòng đúng
    all_correct_versions = master_df[is_correct_condition]
    
    # Lấy danh sách các khóa duy nhất của các conference đúng
    correct_keys_df = all_correct_versions.drop_duplicates(subset=key_cols, keep='first')
    correct_keys_set = set(zip(correct_keys_df['key_title'], correct_keys_df['key_acronym']))
    print(f"Tìm thấy {len(correct_keys_set)} conference có ít nhất một phiên bản đúng.")

    # --- BƯỚC 3: TẠO FILE CORRECT CUỐI CÙNG ---
    print("\n--- BƯỚC 3: Tạo file correct cuối cùng ---")
    # Từ tất cả các phiên bản đúng, giữ lại phiên bản mới nhất (xuất hiện đầu tiên) cho mỗi conference
    final_correct_df = all_correct_versions.drop_duplicates(subset=key_cols, keep='first')
    
    # --- BƯỚC 4: TẠO FILE RECRAWL CUỐI CÙNG ---
    print("\n--- BƯỚC 4: Tạo file recrawl cuối cùng ---")
    # Lấy tất cả các dòng có note
    all_recrawl_versions = master_df[~is_correct_condition].copy()
    
    if not all_recrawl_versions.empty:
        # Loại bỏ những conference đã có trong danh sách đúng
        mask_keep = ~all_recrawl_versions.apply(
            lambda row: (row['key_title'], row['key_acronym']) in correct_keys_set,
            axis=1
        )
        recrawl_to_keep = all_recrawl_versions[mask_keep]
        
        # Từ những dòng còn lại, giữ lại phiên bản mới nhất
        final_recrawl_df = recrawl_to_keep.drop_duplicates(subset=key_cols, keep='first')
    else:
        final_recrawl_df = pd.DataFrame(columns=master_df.columns)

    # --- BƯỚC 5: LƯU FILE ---
    print("\n--- BƯỚC 5: Hoàn tất và lưu file tổng hợp cuối cùng ---")
    
    # Dọn dẹp cột khóa trước khi lưu
    final_correct_df = final_correct_df.drop(columns=key_cols, errors='ignore')
    final_recrawl_df = final_recrawl_df.drop(columns=key_cols, errors='ignore')

    # Lưu file CORRECT
    final_correct_df.to_csv(aggregated_correct_path, index=False, encoding='utf-8-sig')
    print(f"Đã lưu tổng cộng {len(final_correct_df)} dòng vào file correct '{aggregated_correct_path}'.")

    # Lưu file RECRAWL
    final_recrawl_df.to_csv(aggregated_recrawl_path, index=False, encoding='utf-8-sig')
    print(f"Đã lưu tổng cộng {len(final_recrawl_df)} dòng vào file recrawl '{aggregated_recrawl_path}'.")


# --- Cách sử dụng ---
if __name__ == "__main__":
    input_csv_files_for_note_check = [


        './src/conference/evaluate/batch2.csv',
        './src/conference/evaluate/batch3.csv',
        './src/conference/evaluate/batch8.csv',
        './src/conference/evaluate/batch12.csv',
        './src/conference/evaluate/batch13.csv',
        './src/conference/evaluate/batch16.csv',
        './src/conference/evaluate/batch19.csv',
        './src/conference/evaluate/evaluate_recrawl_batch_2_3_lan_1.csv',
        './src/conference/evaluate/evaluate_recrawl_batch_8_lan_1.csv',
        './src/conference/evaluate/evaluate_recrawl_batch_12_lan_1.csv',
        './src/conference/evaluate/evaluate_recrawl_batch_13_lan_1.csv',
        './src/conference/evaluate/evaluate_recrawl_batch_16_lan_1.csv',
        './src/conference/evaluate/evaluate_recrawl_batch_19_lan_1.csv',
        './src/conference/evaluate/evaluate_recrawl_all_tung_1_50.csv',
        './src/conference/evaluate/evaluate_recrawl_all_tung_51_100.csv',
        './src/conference/evaluate/evaluate_recrawl_all_tung_101_150.csv',
        './src/conference/evaluate/evaluate_recrawl_all_tung_151_159.csv',
        './src/conference/evaluate/evaluate_recrawl_all_tung_1_50_lan_2.csv',
        './src/conference/evaluate/evaluate_recrawl_all_tung_51_100_lan_2.csv',
        './src/conference/evaluate/evaluate_recrawl_all_tung_101_139_lan_2.csv',
        './src/conference/evaluate/evaluate_recrawl_all_tung_lan_3.csv',
        './src/conference/evaluate/evaluate_recrawl_all_tung_1_50_lan_4.csv',
        './src/conference/evaluate/evaluate_crawl_not_crawl_1_50.csv',
        './src/conference/evaluate/evaluate_crawl_not_crawl_51_end.csv',



        
        './src/conference/evaluate/batch4.csv',
        './src/conference/evaluate/batch5.csv',
        './src/conference/evaluate/batch9.csv',
        './src/conference/evaluate/batch14.csv',
        './src/conference/evaluate/batch15.csv',
        './src/conference/evaluate/batch18.csv',
        './src/conference/evaluate/batch20.csv',
        './src/conference/evaluate/recrawl_all_thang_1_50_lan_1_check.csv',
        './src/conference/evaluate/recrawl_all_thang_51_58_lan_1_check.csv',
        
        './src/conference/evaluate/evaluate_recrawl_all_tri_1_50_lan_5.csv',
        './src/conference/evaluate/evaluate_recrawl_all_tri_51_100_lan_5.csv',
        './src/conference/evaluate/evaluate_recrawl_all_tri_101_120_lan_5.csv',
        
    ]
    recrawled_results_files = [

        './src/conference/evaluate/evaluate_recrawl_batch_2_3_lan_1.csv',
        './src/conference/evaluate/evaluate_recrawl_batch_8_lan_1.csv',
        './src/conference/evaluate/evaluate_recrawl_batch_12_lan_1.csv',
        './src/conference/evaluate/evaluate_recrawl_batch_13_lan_1.csv',
        './src/conference/evaluate/evaluate_recrawl_batch_16_lan_1.csv',
        './src/conference/evaluate/evaluate_recrawl_batch_19_lan_1.csv',
        './src/conference/evaluate/evaluate_recrawl_all_tung_1_50.csv',
        './src/conference/evaluate/evaluate_recrawl_all_tung_51_100.csv',
        './src/conference/evaluate/evaluate_recrawl_all_tung_101_150.csv',
        './src/conference/evaluate/evaluate_recrawl_all_tung_151_159.csv',
        './src/conference/evaluate/evaluate_recrawl_all_tung_1_50_lan_2.csv',
        './src/conference/evaluate/evaluate_recrawl_all_tung_51_100_lan_2.csv',
        './src/conference/evaluate/evaluate_recrawl_all_tung_101_139_lan_2.csv',
        './src/conference/evaluate/evaluate_recrawl_all_tung_lan_3.csv',
        './src/conference/evaluate/evaluate_recrawl_all_tung_1_50_lan_4.csv',
        './src/conference/evaluate/evaluate_recrawl_all_tung_1_50_lan_4.csv',


        './src/conference/evaluate/evaluate_crawl_not_crawl_1_50.csv',
        './src/conference/evaluate/evaluate_crawl_not_crawl_51_end.csv',


        
        './src/conference/evaluate/recrawl_batch_4_check_lan_1.csv',
        './src/conference/evaluate/recrawl_batch_5_check_lan_1.csv',
        './src/conference/evaluate/recrawl_batch_9_check_lan_1.csv',
        './src/conference/evaluate/recrawl_batch_14_check_lan_1.csv',
        './src/conference/evaluate/recrawl_batch_15_check_lan_1.csv',
        './src/conference/evaluate/recrawl_batch_18_check_lan_1.csv',
        './src/conference/evaluate/recrawl_batch_20_check_lan_1.csv',
        './src/conference/evaluate/recrawl_all_tri_1_50.csv',
        './src/conference/evaluate/recrawl_all_tri_51_100.csv',
        './src/conference/evaluate/recrawl_all_tri_101_150.csv',
        './src/conference/evaluate/recrawl_all_tri_1_32_lan_2.csv',
        './src/conference/evaluate/recrawl_all_tri_lan_3.csv',
        './src/conference/evaluate/recrawl_all_tri_lan_4.csv',

        './src/conference/evaluate/evaluate_recrawl_all_tri_1_50_lan_5.csv',
        './src/conference/evaluate/evaluate_recrawl_all_tri_51_100_lan_5.csv',
        './src/conference/evaluate/evaluate_recrawl_all_tri_101_120_lan_5.csv',

    ]
    aggregated_correct_file_note = './src/conference/evaluate/tri_tung_all_correct_final.csv'
    aggregated_recrawl_file_note = './src/conference/evaluate/tri_tung_all_recrawl_final.csv'

    all_files_for_counting = recrawled_results_files + input_csv_files_for_note_check
    calculate_and_print_total_unique_count(all_files_for_counting)

    process_by_any_correct_version(
        input_files=input_csv_files_for_note_check,
        aggregated_correct_path=aggregated_correct_file_note,
        aggregated_recrawl_path=aggregated_recrawl_file_note,
        recrawled_files=recrawled_results_files
    )


 BƯỚC 0: TÍNH TOÁN TỔNG SỐ CONFERENCE DUY NHẤT TRÊN TẤT CẢ CÁC FILE
Đã xử lý file để đếm: evaluate_recrawl_batch_2_3_lan_1.csv
Đã xử lý file để đếm: evaluate_recrawl_batch_8_lan_1.csv
Đã xử lý file để đếm: evaluate_recrawl_batch_12_lan_1.csv
Đã xử lý file để đếm: evaluate_recrawl_batch_13_lan_1.csv
Đã xử lý file để đếm: evaluate_recrawl_batch_16_lan_1.csv
Đã xử lý file để đếm: evaluate_recrawl_batch_19_lan_1.csv
Đã xử lý file để đếm: evaluate_recrawl_all_tung_1_50.csv
Đã xử lý file để đếm: evaluate_recrawl_all_tung_51_100.csv
Đã xử lý file để đếm: evaluate_recrawl_all_tung_101_150.csv
Đã xử lý file để đếm: evaluate_recrawl_all_tung_151_159.csv
Đã xử lý file để đếm: evaluate_recrawl_all_tung_1_50_lan_2.csv
Đã xử lý file để đếm: evaluate_recrawl_all_tung_51_100_lan_2.csv
Đã xử lý file để đếm: evaluate_recrawl_all_tung_101_139_lan_2.csv
Đã xử lý file để đếm: evaluate_recrawl_all_tung_lan_3.csv
Đã xử lý file để đếm: evaluate_recrawl_all_tung_1_50_lan_4.csv
Đã xử lý file để đếm: evaluate_r

In [3]:
import pandas as pd
import os
import json

def merge_csv_common_columns_ordered(file_paths, output_filename):
    """
    Đọc nhiều file CSV, tìm các cột chung (phân biệt hoa thường),
    sau đó hợp nhất các file chỉ với các cột chung đó.
    Thứ tự các cột trong file kết quả sẽ theo thứ tự của các cột chung
    trong file CSV cuối cùng trong danh sách input.

    Args:
        file_paths (list): Danh sách các đường dẫn đầy đủ đến các file CSV.
        output_filename (str): Đường dẫn và tên file CSV đầu ra.
    
    Returns:
        bool: True nếu merge thành công, False nếu có lỗi.
    """
    if not file_paths:
        print("Lỗi: Danh sách đường dẫn file CSV trống. Không có gì để xử lý.")
        return False

    dataframes = []
    # Bước 1: Đọc tất cả các file CSV vào DataFrame
    for fp in file_paths:
        try:
            # Thêm dtype=str để đảm bảo mọi thứ được đọc vào dưới dạng chuỗi,
            # tránh việc pandas tự động chuyển đổi kiểu dữ liệu có thể làm hỏng JSON.
            df = pd.read_csv(fp, dtype=str)
            dataframes.append(df)
            print(f"Đã đọc thành công: {fp} (có {len(df.columns)} cột)")
        except FileNotFoundError:
            print(f"Lỗi: Không tìm thấy file tại đường dẫn: {fp}. Bỏ qua file này.")
            continue
        except pd.errors.EmptyDataError:
            print(f"Cảnh báo: File trống hoặc không có header: {fp}. Bỏ qua file này.")
            continue
        except Exception as e:
            print(f"Lỗi khi đọc file {fp}: {e}. Bỏ qua file này.")
            continue

    if not dataframes:
        print("Không có file CSV nào được đọc thành công. Không thể thực hiện merge.")
        return False

    # Bước 2: Tìm danh sách các cột chung có ở tất cả các file
    common_columns_set = set(dataframes[0].columns)
    for i in range(1, len(dataframes)):
        common_columns_set.intersection_update(set(dataframes[i].columns))

    if not common_columns_set:
        print("Không tìm thấy cột chung nào ở TẤT CẢ các file CSV đã đọc. Không tạo file mới.")
        return False

    # Bước 3: Lấy thứ tự cột từ file cuối cùng trong danh sách
    last_df = dataframes[-1]
    ordered_common_columns = [col for col in last_df.columns if col in common_columns_set]
    
    if len(ordered_common_columns) != len(common_columns_set):
        print("Cảnh báo: Thứ tự cột từ file cuối cùng không chứa tất cả các cột chung. Sẽ sắp xếp theo alphabet.")
        ordered_common_columns = sorted(list(common_columns_set))

    print(f"\nTìm thấy {len(ordered_common_columns)} cột chung ở tất cả các file:")
    print(f"Thứ tự cột trong file output sẽ dựa trên file cuối cùng: {file_paths[-1]}")
    print(f"Thứ tự các cột: {ordered_common_columns}")

    # Bước 4: Tạo danh sách các DataFrame mới, chỉ chứa các cột chung
    filtered_dataframes = [df[ordered_common_columns] for df in dataframes]

    # Bước 5: Hợp nhất tất cả các DataFrame đã lọc lại với nhau
    try:
        merged_df = pd.concat(filtered_dataframes, ignore_index=True)

        # Bước 6: Lưu DataFrame đã hợp nhất vào file CSV mới
        merged_df.to_csv(output_filename, index=False, encoding='utf-8-sig')
        print(f"\nĐã hợp nhất thành công các file vào '{output_filename}'.")
        print(f"File kết quả có {merged_df.shape[0]} dòng và {merged_df.shape[1]} cột.")
        return True
    except Exception as e:
        print(f"Lỗi khi hợp nhất các DataFrame hoặc ghi file: {e}")
        return False

def validate_json_in_date_columns(csv_filepath):
    """
    Kiểm tra các cột kết thúc bằng "Date" trong một file CSV để xem mỗi ô
    có chứa một chuỗi JSON hợp lệ hay không.

    Args:
        csv_filepath (str): Đường dẫn đến file CSV cần kiểm tra.
    """
    print("\n--- Bắt đầu kiểm tra định dạng JSON trong các cột 'Date' ---")
    try:
        df = pd.read_csv(csv_filepath, dtype=str).fillna('') # Đọc mọi thứ dạng chuỗi, và thay NaN bằng chuỗi rỗng
    except FileNotFoundError:
        print(f"Lỗi: Không tìm thấy file '{csv_filepath}' để kiểm tra.")
        return
    except Exception as e:
        print(f"Lỗi khi đọc file '{csv_filepath}': {e}")
        return

    # Tìm các cột có tên kết thúc bằng "Date"
    date_columns = [col for col in df.columns if col.endswith("Date")]

    if not date_columns:
        print("Không tìm thấy cột nào kết thúc bằng 'Date' để kiểm tra.")
        return

    print(f"Các cột sẽ được kiểm tra: {date_columns}")

    invalid_entries = []
    # Lặp qua từng cột cần kiểm tra
    for col_name in date_columns:
        # Lặp qua từng dòng (index và value) trong cột đó
        for index, value in df[col_name].items():
            # Bỏ qua các ô rỗng hoặc chỉ có khoảng trắng
            if not value or value.isspace():
                continue

            try:
                # Thử phân tích chuỗi thành JSON
                json.loads(value)
            except json.JSONDecodeError:
                # Nếu thất bại, ghi lại thông tin lỗi
                invalid_entries.append({
                    "row_index": index,
                    "column": col_name,
                    "value": value
                })

    # In kết quả
    if not invalid_entries:
        print("\n>>> KIỂM TRA THÀNH CÔNG: Tất cả các giá trị trong các cột 'Date' đều là JSON hợp lệ (hoặc rỗng).")
    else:
        print(f"\n>>> KIỂM TRA THẤT BẠI: Tìm thấy {len(invalid_entries)} giá trị không phải là JSON hợp lệ:")
        print("-" * 50)
        for error in invalid_entries:
            print(f"  - Dòng (chỉ số 0): {error['row_index']}")
            print(f"    Cột              : '{error['column']}'")
            print(f"    Giá trị không hợp lệ: {error['value']}")
            print("-" * 20)

# --- HÀM ĐÃ ĐƯỢC CẬP NHẬT ---
def create_missing_info_file(full_csv_path, output_missing_info_path):
    """
    Đọc file CSV đã hợp nhất, lọc ra các dòng có 'conferenceDates' hoặc 'location' trống
    hoặc chứa các giá trị cụ thể biểu thị thông tin thiếu,
    và lưu các cột 'title', 'acronym' của chúng vào một file mới.

    Args:
        full_csv_path (str): Đường dẫn đến file CSV đầy đủ (đã merge).
        output_missing_info_path (str): Đường dẫn file CSV đầu ra cho dữ liệu thiếu.
    """
    print("\n--- Bắt đầu tạo file chứa các dòng thiếu thông tin ---")
    try:
        # Đọc file full.csv, thay thế các giá trị NaN bằng chuỗi rỗng để dễ xử lý
        df = pd.read_csv(full_csv_path, dtype=str).fillna('')
    except FileNotFoundError:
        print(f"Lỗi: Không tìm thấy file '{full_csv_path}' để xử lý.")
        return
    except Exception as e:
        print(f"Lỗi khi đọc file '{full_csv_path}': {e}")
        return

    # Kiểm tra xem các cột cần thiết có tồn tại không
    required_columns = ['conferenceDates', 'location', 'title', 'acronym']
    if not all(col in df.columns for col in required_columns):
        print(f"Lỗi: File '{full_csv_path}' thiếu một hoặc nhiều cột cần thiết: {required_columns}.")
        return

    # Định nghĩa các giá trị được coi là thiếu thông tin
    missing_dates_values = ['', 'To be announced', 'TBD', 'To be determined']
    missing_location_values = ['', 'No location', 'TBD']

    # Lọc các dòng mà 'conferenceDates' hoặc 'location' nằm trong danh sách các giá trị thiếu
    missing_info_df = df[(df['conferenceDates'].isin(missing_dates_values)) | 
                         (df['location'].isin(missing_location_values))]

    if missing_info_df.empty:
        print("Không tìm thấy dòng nào có 'conferenceDates' hoặc 'location' trống/thiếu thông tin cụ thể.")
        return

    # Chỉ giữ lại hai cột 'title' và 'acronym'
    final_df = missing_info_df[['title', 'acronym']]

    # Lưu vào file CSV mới
    try:
        final_df.to_csv(output_missing_info_path, index=False, encoding='utf-8-sig')
        print(f"\nĐã tạo thành công file '{output_missing_info_path}' với {len(final_df)} dòng.")
    except Exception as e:
        print(f"Lỗi khi ghi file '{output_missing_info_path}': {e}")


# --- Cách sử dụng ---
if __name__ == "__main__":
    # Đặt danh sách các đường dẫn đến file CSV của bạn vào đây.
    input_files = [
        './src/conference/evaluate/tri_tung_all_correct_final.csv',
        './src/conference/evaluate/correctFiles/batch_1_6_7_10_11_17.csv'
    ]

    # Đặt tên cho các file CSV kết quả đầu ra
    output_merged_file = './src/conference/evaluate/full.csv'
    output_missing_file = './src/conference/evaluate/missing_info.csv' # Tên file mới

    # Gọi hàm để thực hiện việc merge
    print("Bắt đầu quá trình merge các file CSV...")
    merge_successful = merge_csv_common_columns_ordered(input_files, output_merged_file)

    # Nếu việc merge thành công, thực hiện các bước tiếp theo
    if merge_successful:
        # 1. Kiểm tra định dạng JSON trong file kết quả
        validate_json_in_date_columns(output_merged_file)
        
        # 2. Tạo file chứa các dòng thiếu thông tin
        create_missing_info_file(output_merged_file, output_missing_file)

    print("\n--- Hoàn thành chương trình ---")

Bắt đầu quá trình merge các file CSV...
Đã đọc thành công: ./src/conference/evaluate/tri_tung_all_correct_final.csv (có 39 cột)
Đã đọc thành công: ./src/conference/evaluate/correctFiles/batch_1_6_7_10_11_17.csv (có 25 cột)

Tìm thấy 25 cột chung ở tất cả các file:
Thứ tự cột trong file output sẽ dựa trên file cuối cùng: ./src/conference/evaluate/correctFiles/batch_1_6_7_10_11_17.csv
Thứ tự các cột: ['requestId', 'originalRequestId', 'title', 'acronym', 'mainLink', 'cfpLink', 'impLink', 'information', 'conferenceDates', 'year', 'location', 'cityStateProvince', 'country', 'continent', 'type', 'submissionDate', 'notificationDate', 'cameraReadyDate', 'registrationDate', 'otherDate', 'topics', 'publisher', 'summary', 'callForPapers', 'Unnamed: 24']

Đã hợp nhất thành công các file vào './src/conference/evaluate/full.csv'.
File kết quả có 869 dòng và 25 cột.

--- Bắt đầu kiểm tra định dạng JSON trong các cột 'Date' ---
Các cột sẽ được kiểm tra: ['submissionDate', 'notificationDate', 'cameraR

In [4]:
import pandas as pd
import re

def normalize_text(text):
    """
    Chuẩn hóa văn bản bằng cách loại bỏ nội dung bên trong cặp dấu ngoặc đơn
    và chính cặp dấu ngoặc đơn, đồng thời đảm bảo chỉ có một khoảng trắng
    giữa các từ.
    """
    if pd.isna(text):
        return ""

    # Bước 1: Loại bỏ nội dung trong ngoặc đơn và các khoảng trắng xung quanh
    # r'\s*\(.*?\)\s*' sẽ khớp với 0 hoặc nhiều khoảng trắng trước (, nội dung, và 0 hoặc nhiều khoảng trắng sau )
    cleaned_text = re.sub(r'\s*\(.*?\)\s*', ' ', str(text)) # Thay thế bằng MỘT khoảng trắng

    # Bước 2: Chuẩn hóa khoảng trắng: thay thế nhiều khoảng trắng bằng một khoảng trắng duy nhất
    # và loại bỏ khoảng trắng ở đầu/cuối.
    # re.sub(r'\s+', ' ', cleaned_text): thay thế 1 hoặc nhiều khoảng trắng bằng 1 khoảng trắng
    # .strip(): loại bỏ khoảng trắng ở đầu và cuối chuỗi
    final_text = re.sub(r'\s+', ' ', cleaned_text).strip()

    return final_text

def find_unique_in_core_detailed(full_csv_path, core_csv_path):
    """
    Tìm các dòng chỉ có trong file CORE_2023.csv dựa trên cặp (title, acronym)
    sau khi chuẩn hóa, và in ra các bước chi tiết.

    Args:
        full_csv_path (str): Đường dẫn đến file full.csv.
        core_csv_path (str): Đường dẫn đến file CORE_2023.csv.

    Returns:
        pandas.DataFrame: DataFrame chứa các dòng chỉ có trong CORE_2023.csv.
    """

    print("--- BƯỚC 1: ĐỌC DỮ LIỆU ---")
    print(f"Đang đọc file full.csv từ: {full_csv_path}")
    try:
        df_full = pd.read_csv(full_csv_path)
        if 'title' not in df_full.columns or 'acronym' not in df_full.columns:
            raise ValueError("File full.csv phải có các cột 'title' và 'acronym'.")
        print("Đã đọc full.csv thành công. 5 dòng đầu tiên:")
        print(df_full.head().to_string()) # to_string() để in toàn bộ cột
        print(f"Tổng số dòng trong full.csv: {len(df_full)}")
    except FileNotFoundError:
        print(f"Lỗi: Không tìm thấy file full.csv tại đường dẫn: {full_csv_path}")
        return pd.DataFrame()
    except Exception as e:
        print(f"Lỗi khi đọc full.csv: {e}")
        return pd.DataFrame()

    print(f"\nĐang đọc file CORE_2023.csv từ: {core_csv_path}")
    try:
        df_core = pd.read_csv(core_csv_path, header=None)
        if df_core.shape[1] < 3:
            raise ValueError("File CORE_2023.csv phải có ít nhất 3 cột để lấy title và acronym.")
        df_core.rename(columns={1: 'title', 2: 'acronym'}, inplace=True)
        print("Đã đọc CORE_2023.csv thành công. 5 dòng đầu tiên (cột 1 là title, cột 2 là acronym):")
        print(df_core.head().to_string())
        print(f"Tổng số dòng trong CORE_2023.csv: {len(df_core)}")
    except FileNotFoundError:
        print(f"Lỗi: Không tìm thấy file CORE_2023.csv tại đường dẫn: {core_csv_path}")
        return pd.DataFrame()
    except Exception as e:
        print(f"Lỗi khi đọc CORE_2023.csv: {e}")
        return pd.DataFrame()

    print("\n--- BƯỚC 2: CHUẨN HÓA DỮ LIỆU ---")
    print("Áp dụng hàm normalize_text cho cột 'title' và 'acronym' của cả hai DataFrame.")

    df_full['normalized_title'] = df_full['title'].apply(normalize_text)
    df_full['normalized_acronym'] = df_full['acronym'].apply(normalize_text)
    print("\n5 dòng đầu tiên của full.csv sau khi chuẩn hóa:")
    print(df_full[['title', 'normalized_title', 'acronym', 'normalized_acronym']].head().to_string())

    df_core['normalized_title'] = df_core['title'].apply(normalize_text)
    df_core['normalized_acronym'] = df_core['acronym'].apply(normalize_text)
    print("\n5 dòng đầu tiên của CORE_2023.csv sau khi chuẩn hóa:")
    print(df_core[['title', 'normalized_title', 'acronym', 'normalized_acronym']].head().to_string())

    print("\n--- BƯỚC 3: TẠO KHÓA SO SÁNH (KEY) ---")
    print("Tạo cột 'key' từ cặp (normalized_title, normalized_acronym) cho cả hai DataFrame.")
    df_full['key'] = list(zip(df_full['normalized_title'], df_full['normalized_acronym']))
    df_core['key'] = list(zip(df_core['normalized_title'], df_core['normalized_acronym']))

    print("\n5 dòng đầu tiên của full.csv với cột 'key':")
    print(df_full[['title', 'acronym', 'key']].head().to_string())
    print("\n5 dòng đầu tiên của CORE_2023.csv với cột 'key':")
    print(df_core[['title', 'acronym', 'key']].head().to_string())

    print("\n--- BƯỚC 4: TẠO TẬP HỢP CÁC KEY TỪ full.csv ---")
    keys_in_full = set(df_full['key'])
    print(f"Tổng số key duy nhất trong full.csv: {len(keys_in_full)}")
    # In ra một vài key mẫu để kiểm tra
    print("Một vài key mẫu từ full.csv (đã chuẩn hóa):")
    for i, key in enumerate(list(keys_in_full)[:5]):
        print(f"  - {key}")

    print("\n--- BƯỚC 5: LỌC CÁC DÒNG CHỈ CÓ TRONG CORE_2023.csv ---")
    print("Lọc df_core để tìm các dòng mà 'key' của chúng KHÔNG có trong tập hợp key của full.csv.")
    unique_in_core_df = df_core[~df_core['key'].isin(keys_in_full)].copy()

    print(f"\nTìm thấy {len(unique_in_core_df)} dòng chỉ có trong CORE_2023.csv.")
    if not unique_in_core_df.empty:
        print("5 dòng đầu tiên của kết quả (các dòng chỉ có trong CORE_2023.csv):")
        print(unique_in_core_df.head().to_string())
    else:
        print("Không tìm thấy dòng nào chỉ có trong CORE_2023.csv.")

    print("\n--- BƯỚC 6: DỌN DẸP CÁC CỘT TẠM THỜI ---")
    # Xóa các cột tạm thời đã tạo
    unique_in_core_df.drop(columns=['normalized_title', 'normalized_acronym', 'key'], inplace=True, errors='ignore')

    print("Đã xóa các cột 'normalized_title', 'normalized_acronym', 'key' khỏi DataFrame kết quả.")
    print("Quá trình so sánh hoàn tất.")

    return unique_in_core_df

if __name__ == "__main__":
    # --- Cấu hình đường dẫn file của bạn ---
    FULL_CSV_FILE = './src/conference/evaluate/full.csv'
    CORE_CSV_FILE = './src/conference/csv/CORE_2023.csv'
    OUTPUT_CSV_FILE = './src/conference/evaluate/recrawl_all_core.csv'

    # Gọi hàm để tìm các dòng duy nhất với chi tiết
    result_df = find_unique_in_core_detailed(FULL_CSV_FILE, CORE_CSV_FILE)

    if not result_df.empty:
        print(f"\n--- KẾT QUẢ CUỐI CÙNG ---")
        print(f"Tổng số dòng duy nhất trong CORE_2023.csv: {len(result_df)}")
        print(f"Kết quả đã được lưu vào file: {OUTPUT_CSV_FILE}")
        result_df.to_csv(OUTPUT_CSV_FILE, index=False)
    else:
        print("\n--- KẾT QUẢ CUỐI CÙNG ---")
        print("Không tìm thấy dòng nào chỉ có trong CORE_2023.csv.")

--- BƯỚC 1: ĐỌC DỮ LIỆU ---
Đang đọc file full.csv từ: ./src/conference/evaluate/full.csv
Đã đọc full.csv thành công. 5 dòng đầu tiên:
                      requestId originalRequestId                                                                  title  acronym                                           mainLink                                                    cfpLink                                            impLink                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               

In [8]:
import pandas as pd
import re

# Sử dụng lại hàm normalize_text đã được cải tiến
def normalize_text(text):
    """
    Chuẩn hóa văn bản bằng cách loại bỏ nội dung bên trong cặp dấu ngoặc đơn
    và chính cặp dấu ngoặc đơn, đồng thời đảm bảo chỉ có một khoảng trắng
    giữa các từ.
    """
    if pd.isna(text):
        return ""

    cleaned_text = re.sub(r'\s*\(.*?\)\s*', ' ', str(text))
    final_text = re.sub(r'\s+', ' ', cleaned_text).strip()

    return final_text

def filter_recrawl_list_with_non_links(recrawl_all_core_path, crawling_files_paths, non_link_acronyms_path, output_csv_path):
    """
    Lọc danh sách recrawl_all_core.csv bằng cách loại bỏ các conference
    đang được crawl (dựa trên 3 file CSV khác) và các conference
    có acronym trong danh sách non-link từ file TXT.

    Args:
        recrawl_all_core_path (str): Đường dẫn đến file recrawl_all_core.csv.
        crawling_files_paths (list): Danh sách các đường dẫn đến các file CSV
                                      chứa danh sách conference đang được crawl.
        non_link_acronyms_path (str): Đường dẫn đến file TXT chứa các acronym non-link.
        output_csv_path (str): Đường dẫn để lưu file CSV kết quả cuối cùng.
    """
    print("--- BƯỚC 1: ĐỌC DANH SÁCH CẦN RECRAWL TỪ recrawl_all_core.csv ---")
    try:
        df_recrawl = pd.read_csv(recrawl_all_core_path)
        if 'title' not in df_recrawl.columns or 'acronym' not in df_recrawl.columns:
            raise ValueError(f"File {recrawl_all_core_path} phải có các cột 'title' và 'acronym'.")
        print(f"Đã đọc {len(df_recrawl)} dòng từ {recrawl_all_core_path}.")
    except FileNotFoundError:
        print(f"Lỗi: Không tìm thấy file: {recrawl_all_core_path}")
        return
    except Exception as e:
        print(f"Lỗi khi đọc {recrawl_all_core_path}: {e}")
        return

    print("\n--- BƯỚC 2: CHUẨN HÓA VÀ TẠO KEY CHO DANH SÁCH RECRAWL ---")
    df_recrawl['normalized_title'] = df_recrawl['title'].apply(normalize_text)
    df_recrawl['normalized_acronym'] = df_recrawl['acronym'].apply(normalize_text)
    df_recrawl['key'] = list(zip(df_recrawl['normalized_title'], df_recrawl['normalized_acronym']))
    recrawl_keys = set(df_recrawl['key'])
    print(f"Tổng số key duy nhất trong recrawl_all_core: {len(recrawl_keys)}")

    print("\n--- BƯỚC 3: ĐỌC VÀ TỔNG HỢP CÁC CONFERENCE ĐANG ĐƯỢC CRAWL ---")
    all_crawling_keys = set()
    for i, file_path in enumerate(crawling_files_paths):
        print(f"Đang đọc file crawling {i+1}/{len(crawling_files_paths)}: {file_path}")
        try:
            df_crawling = pd.read_csv(file_path)
            if 'title' not in df_crawling.columns or 'acronym' not in df_crawling.columns:
                print(f"Cảnh báo: File {file_path} không có cột 'title' hoặc 'acronym'. Bỏ qua file này.")
                continue

            df_crawling['normalized_title'] = df_crawling['title'].apply(normalize_text)
            df_crawling['normalized_acronym'] = df_crawling['acronym'].apply(normalize_text)
            df_crawling['key'] = list(zip(df_crawling['normalized_title'], df_crawling['normalized_acronym']))

            all_crawling_keys.update(set(df_crawling['key']))
            print(f"  Đã thêm {len(set(df_crawling['key']))} key từ {file_path}. Tổng số key đang crawl: {len(all_crawling_keys)}")

        except FileNotFoundError:
            print(f"Lỗi: Không tìm thấy file crawling tại đường dẫn: {file_path}. Bỏ qua file này.")
        except Exception as e:
            print(f"Lỗi khi đọc file crawling {file_path}: {e}. Bỏ qua file này.")

    print(f"\nTổng số key duy nhất từ tất cả các file đang crawl: {len(all_crawling_keys)}")

    print("\n--- BƯỚC 4: ĐỌC DANH SÁCH ACRONYM NON-LINK ---")
    non_link_acronyms = set()
    try:
        with open(non_link_acronyms_path, 'r', encoding='utf-8') as f:
            for line in f:
                acronym = line.strip() # Loại bỏ khoảng trắng và ký tự xuống dòng
                if acronym: # Đảm bảo không thêm dòng trống
                    non_link_acronyms.add(normalize_text(acronym)) # Chuẩn hóa acronym trước khi thêm
        print(f"Đã đọc {len(non_link_acronyms)} acronym non-link từ {non_link_acronyms_path}.")
    except FileNotFoundError:
        print(f"Lỗi: Không tìm thấy file non-link acronyms tại đường dẫn: {non_link_acronyms_path}. Bỏ qua bước lọc này.")
    except Exception as e:
        print(f"Lỗi khi đọc file non-link acronyms: {e}. Bỏ qua bước lọc này.")

    print("\n--- BƯỚC 5: LỌC DANH SÁCH RECRAWL CUỐI CÙNG ---")
    print("Tìm các conference trong recrawl_all_core mà KHÔNG có trong danh sách đang crawl")
    print("VÀ KHÔNG có acronym trong danh sách non-link.")

    # Bắt đầu với danh sách recrawl_all_core
    current_recrawl_df = df_recrawl.copy()
    initial_count = len(current_recrawl_df)
    print(f"Số dòng ban đầu trong danh sách recrawl: {initial_count}")

    # Lọc bỏ các conference đang được crawl
    filtered_by_crawling = current_recrawl_df[~current_recrawl_df['key'].isin(all_crawling_keys)].copy()
    print(f"Số dòng sau khi loại bỏ các conference đang crawl: {len(filtered_by_crawling)} (Đã loại bỏ {initial_count - len(filtered_by_crawling)} dòng).")

    # Lọc bỏ các conference có acronym trong danh sách non-link
    # Chúng ta cần chuẩn hóa acronym của df_recrawl trước khi so sánh với non_link_acronyms
    final_recrawl_list_df = filtered_by_crawling[~filtered_by_crawling['normalized_acronym'].isin(non_link_acronyms)].copy()
    print(f"Số dòng sau khi loại bỏ các conference non-link: {len(final_recrawl_list_df)} (Đã loại bỏ {len(filtered_by_crawling) - len(final_recrawl_list_df)} dòng).")


    print(f"\nTìm thấy {len(final_recrawl_list_df)} conference cần recrawl cuối cùng.")

    print("\n--- BƯỚC 6: DỌN DẸP VÀ LƯU KẾT QUẢ ---")
    # Xóa các cột tạm thời đã tạo
    final_recrawl_list_df.drop(columns=['normalized_title', 'normalized_acronym', 'key'], inplace=True, errors='ignore')

    if not final_recrawl_list_df.empty:
        print(f"5 dòng đầu tiên của danh sách recrawl cuối cùng:")
        print(final_recrawl_list_df.head().to_string())
        print(f"\nKết quả đã được lưu vào file: {output_csv_path}")
        final_recrawl_list_df.to_csv(output_csv_path, index=False)
    else:
        print("Không tìm thấy conference nào cần recrawl cuối cùng sau khi lọc.")

if __name__ == "__main__":
    # --- Cấu hình đường dẫn file của bạn ---
    RECRAWL_ALL_CORE_FILE = './src/conference/evaluate/recrawl_all_core.csv'

    # Danh sách các file CSV chứa conference đang được crawl
    # Đảm bảo các file này có cột 'title' và 'acronym'
    CRAWLING_FILES = [
       './src/conference/evaluate/tri_tung_all_recrawl_final.csv'
    ]

    # Đường dẫn đến file TXT chứa các acronym non-link
    NON_LINK_ACRONYMS_FILE = './non_link.txt' # Ví dụ: non_link_acronyms.txt

    OUTPUT_FINAL_RECRAWL_FILE = './src/conference/evaluate/final_recrawl_list.csv'

    # Gọi hàm để lọc danh sách recrawl
    filter_recrawl_list_with_non_links(RECRAWL_ALL_CORE_FILE, CRAWLING_FILES, NON_LINK_ACRONYMS_FILE, OUTPUT_FINAL_RECRAWL_FILE)

--- BƯỚC 1: ĐỌC DANH SÁCH CẦN RECRAWL TỪ recrawl_all_core.csv ---
Đã đọc 87 dòng từ ./src/conference/evaluate/recrawl_all_core.csv.

--- BƯỚC 2: CHUẨN HÓA VÀ TẠO KEY CHO DANH SÁCH RECRAWL ---
Tổng số key duy nhất trong recrawl_all_core: 87

--- BƯỚC 3: ĐỌC VÀ TỔNG HỢP CÁC CONFERENCE ĐANG ĐƯỢC CRAWL ---
Đang đọc file crawling 1/1: ./src/conference/evaluate/tri_tung_all_recrawl_final.csv
  Đã thêm 17 key từ ./src/conference/evaluate/tri_tung_all_recrawl_final.csv. Tổng số key đang crawl: 17

Tổng số key duy nhất từ tất cả các file đang crawl: 17

--- BƯỚC 4: ĐỌC DANH SÁCH ACRONYM NON-LINK ---
Đã đọc 56 acronym non-link từ ./non_link.txt.

--- BƯỚC 5: LỌC DANH SÁCH RECRAWL CUỐI CÙNG ---
Tìm các conference trong recrawl_all_core mà KHÔNG có trong danh sách đang crawl
VÀ KHÔNG có acronym trong danh sách non-link.
Số dòng ban đầu trong danh sách recrawl: 87
Số dòng sau khi loại bỏ các conference đang crawl: 70 (Đã loại bỏ 17 dòng).
Số dòng sau khi loại bỏ các conference non-link: 28 (Đã loại

In [63]:
import pandas as pd

def merge_and_select_columns(full_csv_path, unique_csv_path, output_csv_path):
    """
    Gộp hai file CSV (full.csv và unique_in_CORE_2023.csv) và chỉ giữ lại
    các cột 'title' và 'acronym', sau đó lưu vào một file CSV mới.

    Args:
        full_csv_path (str): Đường dẫn đến file full.csv.
        unique_csv_path (str): Đường dẫn đến file unique_in_CORE_2023.csv.
        output_csv_path (str): Đường dẫn để lưu file CSV kết quả.
    """
    print(f"Đang đọc file full.csv từ: {full_csv_path}")
    try:
        df_full = pd.read_csv(full_csv_path)
        # Đảm bảo các cột 'title' và 'acronym' tồn tại
        if 'title' not in df_full.columns or 'acronym' not in df_full.columns:
            raise ValueError(f"File {full_csv_path} phải có các cột 'title' và 'acronym'.")
        print("Đã đọc full.csv thành công.")
    except FileNotFoundError:
        print(f"Lỗi: Không tìm thấy file full.csv tại đường dẫn: {full_csv_path}")
        return
    except Exception as e:
        print(f"Lỗi khi đọc full.csv: {e}")
        return

    print(f"Đang đọc file unique_in_CORE_2023.csv từ: {unique_csv_path}")
    try:
        df_unique = pd.read_csv(unique_csv_path)
        # Đảm bảo các cột 'title' và 'acronym' tồn tại
        if 'title' not in df_unique.columns or 'acronym' not in df_unique.columns:
            raise ValueError(f"File {unique_csv_path} phải có các cột 'title' và 'acronym'.")
        print("Đã đọc unique_in_CORE_2023.csv thành công.")
    except FileNotFoundError:
        print(f"Lỗi: Không tìm thấy file unique_in_CORE_2023.csv tại đường dẫn: {unique_csv_path}")
        return
    except Exception as e:
        print(f"Lỗi khi đọc unique_in_CORE_2023.csv: {e}")
        return

    print("Đang chọn các cột 'title' và 'acronym' từ cả hai DataFrame...")
    # Chọn chỉ các cột 'title' và 'acronym' từ mỗi DataFrame
    df_full_selected = df_full[['title', 'acronym']]
    df_unique_selected = df_unique[['title', 'acronym']]

    print("Đang gộp hai DataFrame...")
    # Gộp hai DataFrame theo chiều dọc (thêm hàng)
    # ignore_index=True để reset index của DataFrame kết quả
    merged_df = pd.concat([df_full_selected, df_unique_selected], ignore_index=True)

    print(f"Tổng số dòng sau khi gộp: {len(merged_df)}")

    # Tùy chọn: Xóa các dòng trùng lặp nếu bạn muốn một danh sách duy nhất
    # Dòng trùng lặp ở đây có nghĩa là cả title và acronym đều giống hệt nhau
    # Nếu bạn muốn giữ lại tất cả các dòng, kể cả trùng lặp, hãy bỏ qua bước này
    print("Đang loại bỏ các dòng trùng lặp (nếu có)...")
    initial_rows = len(merged_df)
    merged_df.drop_duplicates(inplace=True)
    rows_after_dedup = len(merged_df)
    print(f"Số dòng sau khi loại bỏ trùng lặp: {rows_after_dedup} (Đã loại bỏ {initial_rows - rows_after_dedup} dòng trùng lặp).")


    print(f"Đang lưu kết quả vào file: {output_csv_path}")
    try:
        merged_df.to_csv(output_csv_path, index=False)
        print("Đã lưu file thành công.")
    except Exception as e:
        print(f"Lỗi khi lưu file: {e}")

if __name__ == "__main__":
    FULL_CSV_FILE = './src/conference/evaluate/full.csv'
    UNIQUE_CSV_FILE = './src/conference/evaluate/recrawl_all_core.csv'
    OUTPUT_MERGED_FILE = './src/conference/evaluate/merged_full_and_unique.csv'

    # Gọi hàm để gộp các file
    merge_and_select_columns(FULL_CSV_FILE, UNIQUE_CSV_FILE, OUTPUT_MERGED_FILE)

Đang đọc file full.csv từ: ./src/conference/evaluate/full.csv
Đã đọc full.csv thành công.
Đang đọc file unique_in_CORE_2023.csv từ: ./src/conference/evaluate/recrawl_all_core.csv
Đã đọc unique_in_CORE_2023.csv thành công.
Đang chọn các cột 'title' và 'acronym' từ cả hai DataFrame...
Đang gộp hai DataFrame...
Tổng số dòng sau khi gộp: 956
Đang loại bỏ các dòng trùng lặp (nếu có)...
Số dòng sau khi loại bỏ trùng lặp: 955 (Đã loại bỏ 1 dòng trùng lặp).
Đang lưu kết quả vào file: ./src/conference/evaluate/merged_full_and_unique.csv
Đã lưu file thành công.


In [64]:
import pandas as pd

def find_and_export_duplicates(input_csv_path, output_csv_path, subset_columns=None):
    """
    Tìm và xuất các dòng trùng lặp trong một file CSV.

    Args:
        input_csv_path (str): Đường dẫn đến file CSV đầu vào (ví dụ: full.csv).
        output_csv_path (str): Đường dẫn để lưu file CSV chứa các dòng trùng lặp.
        subset_columns (list, optional): Danh sách các tên cột để kiểm tra trùng lặp.
                                         Nếu None, tất cả các cột sẽ được sử dụng.
                                         Ví dụ: ['title', 'acronym']
    """
    print(f"Đang đọc file CSV từ: {input_csv_path}")
    try:
        df = pd.read_csv(input_csv_path)
        print(f"Đã đọc {len(df)} dòng từ {input_csv_path}.")
    except FileNotFoundError:
        print(f"Lỗi: Không tìm thấy file CSV tại đường dẫn: {input_csv_path}")
        return
    except Exception as e:
        print(f"Lỗi khi đọc file CSV: {e}")
        return

    print("Đang tìm kiếm các dòng trùng lặp...")

    # Tìm các dòng trùng lặp
    # keep=False: Đánh dấu TẤT CẢ các lần xuất hiện của một dòng trùng lặp là True
    #             (bao gồm cả lần xuất hiện đầu tiên).
    #             Nếu muốn chỉ đánh dấu các bản sao (không bao gồm bản gốc đầu tiên),
    #             sử dụng keep='first' hoặc keep='last'.
    # subset: Các cột để kiểm tra trùng lặp. Nếu None, kiểm tra tất cả các cột.
    duplicate_rows = df[df.duplicated(subset=subset_columns, keep=False)]

    if not duplicate_rows.empty:
        print(f"Tìm thấy {len(duplicate_rows)} dòng trùng lặp.")
        print(f"Đang lưu các dòng trùng lặp vào file: {output_csv_path}")
        try:
            duplicate_rows.to_csv(output_csv_path, index=False)
            print("Đã lưu file trùng lặp thành công.")
        except Exception as e:
            print(f"Lỗi khi lưu file trùng lặp: {e}")
    else:
        print("Không tìm thấy dòng trùng lặp nào.")

if __name__ == "__main__":
    # --- Cấu hình đường dẫn file của bạn ---
    INPUT_CSV_FILE = './src/conference/evaluate/full.csv'
    OUTPUT_DUPLICATES_FILE = 'full_duplicates.csv'

    # --- Cấu hình các cột để kiểm tra trùng lặp ---
    # Nếu bạn muốn kiểm tra trùng lặp dựa trên TẤT CẢ các cột, hãy để là None
    # COLUMNS_TO_CHECK = None

    # Nếu bạn muốn kiểm tra trùng lặp chỉ dựa trên 'title' và 'acronym', hãy sử dụng:
    COLUMNS_TO_CHECK = ['title', 'acronym']

    # Gọi hàm để tìm và xuất các dòng trùng lặp
    find_and_export_duplicates(INPUT_CSV_FILE, OUTPUT_DUPLICATES_FILE, COLUMNS_TO_CHECK)

Đang đọc file CSV từ: ./src/conference/evaluate/full.csv
Đã đọc 869 dòng từ ./src/conference/evaluate/full.csv.
Đang tìm kiếm các dòng trùng lặp...
Tìm thấy 2 dòng trùng lặp.
Đang lưu các dòng trùng lặp vào file: full_duplicates.csv
Đã lưu file trùng lặp thành công.
